In [1]:
from sage.all import *
import struct

DR = RealField(53)
DD = RealField(190)

def float_to_hex(f):
    packed = struct.pack('>f', float(f))
    return '0x' + packed.hex()


def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

print(double_to_hex(DD(1)/DD(2)))
print(double_to_hex(DD(5)/DD(24)))
print(double_to_hex(DD(61)/DD(720)))
print(double_to_hex(DD(277)/DD(8064)))
print(double_to_hex(DD(50521)/DD(3628800)))
print(double_to_hex(DD('540553')/DD('95800320')))
print(double_to_hex(DD('199360981')/DD('87178291200')))

0x3fe0000000000000
0x3fcaaaaaaaaaaaab
0x3fb5b05b05b05b06
0x3fa1965965965966
0x3f8c834283cd3723
0x3f771c9634451939
0x3f62bbcea738201f


In [2]:
# test max cutoff for direct evaluation
from mpmath import mp, sec, taylor, mpf

mp.dps = 50 
true_sec = lambda x: sec(x)
approx_sec = lambda x, n: sum(c * mpf(x)**i for i, c in enumerate(taylor(sec, 0, n+1)))

def max_x_for_order(n, max_rel_error=2**-68):
    x = 0.0
    step = 0.001
    while True:
        x += step
        exact = true_sec(x)
        approx = approx_sec(x, n)
        rel_error = abs((approx - exact) / exact)
        if rel_error > max_rel_error:
            return x - step

print("Order 6:", max_x_for_order(6))
print("Order 8:", max_x_for_order(8))
print("Order 10:", max_x_for_order(10))
print("Order 12:", max_x_for_order(12))
print("Order 14:", max_x_for_order(14))

Order 6: 0.004
Order 8: 0.013000000000000005
Order 10: 0.03000000000000002
Order 12: 0.05300000000000004
Order 14: 0.08100000000000006


In [16]:
print(double_to_hex(2**-64))
print(double_to_hex(2**-66))

0x3bf0000000000000
0x3bd0000000000000


In [26]:
x = var('x')
f = 1 / cos(x)

# Generate Taylor (Maclaurin) series for sec(x) around x=0 up to order 10
taylor_series = f.taylor(x, 0, 10)
show(taylor_series)

hi = var('hi')
lo = var('lo')

x = hi + lo

series = 61/720*x**6 + 5/24*x**4 + 1/2*x**2 + 1
expanded_series = series.simplify_full()
print(expanded_series)


50521/3628800*x^10 + 277/8064*x^8 + 61/720*x^6 + 5/24*x^4 + 1/2*x^2 + 1
0.08472222222222223*hi^6 + 0.5083333333333333*hi*lo^5 + 0.08472222222222223*lo^6 + (1.2708333333333335*hi^2 + 0.20833333333333334)*lo^4 + 0.20833333333333334*hi^4 + (1.6944444444444446*hi^3 + 0.8333333333333334*hi)*lo^3 + (1.2708333333333335*hi^4 + 1.25*hi^2 + 0.5)*lo^2 + 0.5*hi^2 + (0.5083333333333333*hi^5 + 0.8333333333333334*hi^3 + 1.0*hi)*lo + 1


In [25]:
# Declare variables
y = var('y')

# Taylor expand sin(y) around y=0 up to y^7
sin_series = sin(y).taylor(y, 0, 7)

u_hi, u_lo = var('u_hi u_lo')
y = u_hi + u_lo

# Substitute y = u_hi + u_lo
sin_series_hi_lo = sin_series.substitute(y = u_hi + u_lo)

# Expand all terms
expanded = sin_series_hi_lo.expand()

# Now collect terms similar to the target form manually:
# Split into hi-only and lo-containing parts

# Extract terms involving only u_hi
hi_only = expanded.coefficient(u_lo, 0)
print(hi_only)

# Extract terms involving u_lo (linear in u_lo only here)
lo_term = (expanded - hi_only).factor()

print(lo_term)

# Now print the terms manually in Horner form
print("sin(y) ≈ ")
print("  u_hi")
print("  + u_hi^3 * (-1/6 + u_hi^2 * (1/120 - u_hi^2 * 1/5040))")
print("  + u_lo * (1 + u_hi^2 * (-1/2 + u_hi^2 / 24))")

# Optional: verify that your manual form equals original
manual = (
    u_hi
    + u_hi**3 * (-1/6 + u_hi**2 * (1/120 - u_hi**2 * (1/5040)))
    + u_lo * (1 + u_hi**2 * (-1/2 + u_hi**2 / 24))
)

print("Difference from true Taylor series:")
print((expanded - manual).simplify_full())


-1/5040*u_hi^7 + 1/120*u_hi^5 - 1/6*u_hi^3 + u_hi
-1/5040*(7*u_hi^6 + 21*u_hi^5*u_lo + 35*u_hi^4*u_lo^2 + 35*u_hi^3*u_lo^3 + 21*u_hi^2*u_lo^4 + 7*u_hi*u_lo^5 + u_lo^6 - 210*u_hi^4 - 420*u_hi^3*u_lo - 420*u_hi^2*u_lo^2 - 210*u_hi*u_lo^3 - 42*u_lo^4 + 2520*u_hi^2 + 2520*u_hi*u_lo + 840*u_lo^2 - 5040)*u_lo
sin(y) ≈ 
  u_hi
  + u_hi^3 * (-1/6 + u_hi^2 * (1/120 - u_hi^2 * 1/5040))
  + u_lo * (1 + u_hi^2 * (-1/2 + u_hi^2 / 24))
Difference from true Taylor series:
-(3.96508223080413e-19)*u_hi^7 - 0.0013888888888888885*u_hi^6*u_lo - 0.001388888888888889*u_hi*u_lo^6 - 0.000198412698412698*u_lo^7 + (-0.004166666666666667*u_hi^2 + 0.008333333333333333)*u_lo^5 + (-0.006944444444444444*u_hi^3 + 0.041666666666666664*u_hi)*u_lo^4 + (-0.006944444444444444*u_hi^4 + 0.08333333333333333*u_hi^2 - 0.16666666666666666)*u_lo^3 + (-0.0041666666666666675*u_hi^5 + 0.08333333333333333*u_hi^3 - 0.5*u_hi)*u_lo^2
